<a href="https://colab.research.google.com/github/briag1/ParsingEval/blob/main/T_test_Early_Stopping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install "ray[tune]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.8 MB/s eta 0:00:00


In [ ]:
import ray
from ray import tune
from hyperopt import hp,Trials
from ray import train, tune
import ray
from ray.tune.search.hyperopt import HyperOptSearch
import numpy as np
import os
import torch
import scipy

best_dict={
    "empirical_mean":0,
    "samples":[],
    "num_samples":0,
    "true_mean":0
}

np.save("/content/best_dict",best_dict)
def eval(config,):
    mean=0
    samples=np.random.randn(100)+config["lr"]
    continue_testing_hypothesis=True
    best_dict=np.load("/content/best_dict.npy",allow_pickle=True).item()
    for num_samples, sample in enumerate(samples):
      mean=mean + 1/(num_samples+1)*(sample-mean)
      if best_dict["num_samples"]>0 and num_samples>0 and continue_testing_hypothesis:
        ttestResult=scipy.stats.ttest_rel(samples[:num_samples+1], best_dict["samples"][:num_samples+1], axis=0, alternative='less', keepdims=False)
        # stop if current config is the current config leads to statistically signficantly lower average performances
        if ttestResult.statistic<0 and ttestResult.pvalue<1e-4:
          result={
            "empirical_mean":mean,
            "samples":samples,
            "num_samples":num_samples,
            "true_mean":config["lr"]
          }
          return result
        # continue the test but stop doing t-test if current config leads to statistically signficantly higher average performances
        if ttestResult.statistic>0 and 1-ttestResult.pvalue<1e-4:
          continue_testing_hypothesis=False

    result={
            "empirical_mean":mean,
            "samples":samples,
            "num_samples":num_samples,
            "true_mean":config["lr"]
      }
    #if empirical mean is better this becomes the new best config to test against
    if best_dict["empirical_mean"]<result["empirical_mean"] :
      np.save("/content/best_dict",result)
    return result
ray.init()

config_dict={"lr": hp.uniform("lr", 0.0, 1)}

algo = HyperOptSearch(space=config_dict, metric="empirical_mean", mode="max", random_state_seed=1)

trainable_with_gpu = tune.with_resources(eval, {"cpu": 1, "gpu": 0})

tuner = tune.Tuner(
        tune.with_parameters(trainable_with_gpu),
        tune_config=tune.TuneConfig(
                                search_alg=algo,
                                max_concurrent_trials=1,
                                num_samples=100,
                                    ),
        run_config=train.RunConfig(
            name="xp_test",
            storage_path="/content/test/",
            verbose=0)
    )
result=tuner.fit()
ray.shutdown()


2024-04-25 10:53:53,642	INFO worker.py:1749 -- Started a local Ray instance.
2024-04-25 10:53:55,496	INFO tune.py:633 -- [output] This will use the new output engine with verbosity 0. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949


+--------------------------------------------------+
| Configuration for experiment     xp_test         |
+--------------------------------------------------+
| Search algorithm                 SearchGenerator |
| Scheduler                        FIFOScheduler   |
| Number of trials                 100             |
+--------------------------------------------------+

View detailed results here: /content/test/xp_test
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-04-25_10-53-49_926204_45375/artifacts/2024-04-25_10-53-55/xp_test/driver_artifacts`


2024-04-25 10:59:24,446	INFO tune.py:1021 -- Wrote the latest version of all result files and experiment state to '/content/test/xp_test' in 0.0647s.


In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/e5-mistral-7b-instruct")
# In case you want to reduce the maximum sequence length:
model.max_seq_length = 4096

queries = [
    "am I very Happy",
    "I am not Happy",
]
documents = [
    "I am full of happiness"
]

query_embeddings = model.encode(queries, prompt_name="sts_query")
document_embeddings = model.encode(documents)

scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/178k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
np.load("/content/best_dict.npy",allow_pickle=True).item()['empirical_mean']

0

In [ ]:
result.get_best_result(metric="empirical_mean",mode="max")

Result(
  metrics={'empirical_mean': 1.0919641437062213, 'samples': array([ 0.17946487,  1.57517709,  0.30391609, -0.17500133,  2.32983109,
        0.55228379,  1.43430057,  2.19464612,  1.96838501,  0.79170352,
        1.08734872,  0.28471891,  0.85834668,  2.35588428,  1.40412508,
        0.47168395, -0.05456889,  1.95851584,  1.19486451,  1.58031419,
        0.33029305,  2.49501263,  2.30555004,  0.76092622,  0.17107635,
        1.79906381,  0.23841142,  1.12368997,  1.47242472,  3.30051935,
        2.16901439,  0.04734584,  0.28071246,  0.11876605, -0.80385083,
        0.41550551,  1.16352488,  0.87152925,  1.77397247,  2.38771041,
        1.32004703,  0.98656391, -0.34109435,  0.64609848,  0.29273361,
        1.68573213,  1.68595866,  0.1996902 ,  0.10803701, -0.05473099,
        0.73296762,  1.13217366,  2.18270031,  1.86120742,  0.961074  ,
        1.11985832,  1.47896274,  1.85311747,  1.92631145,  1.31742586,
        2.51968808,  1.56635723,  0.52398346,  1.0761862 ,  1.433120

In [ ]:
results=result

In [ ]:
num_samples=np.array([results[i].metrics["num_samples"] for i in range(len(results))])
print("mean number of samples per trials: ",num_samples.mean()," std: ",num_samples.std())

mean number of samples per trials:  85.09  std:  26.178271524300456


In [ ]:
true_means=np.array([results[i].metrics["true_mean"] for i in range(len(results))])

In [ ]:
true_means.max()

0.9994762891493268

In [ ]:
np.where(true_means>result.get_best_result(metric="empirical_mean",mode="max").metrics['true_mean'])

(array([ 4,  6,  9, 22, 23, 24, 26, 28, 30, 33, 34, 38, 41, 49, 53, 54, 60,
        63, 64, 66, 67, 68, 69, 70, 72, 73, 74, 80, 89, 90, 92, 95]),)

In [ ]:
[results[i].metrics["num_samples"] for i in [ 4,  6,  9, 22, 23, 24, 26, 28, 30, 33, 34, 38, 41, 49, 53, 54, 60,
        63, 64, 66, 67, 68, 69, 70, 72, 73, 74, 80, 89, 90, 92, 95]]

[99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 99,
 3,
 99,
 99,
 99,
 99,
 99,
 99]

In [ ]:
true_means[[30, 42, 95]]

array([0.99975769, 0.99997137, 0.99978522])

In [ ]:
np.load("/content/best_dict.npy",allow_pickle=True)

array({'empirical_mean': 1.5380164995458228, 'samples': array([1.52615117, 1.71281534, 1.26158482, 1.39364789, 1.78971702,
       1.43051631, 1.64033717, 1.95525392, 1.20506222, 1.79177712,
       1.26295632, 1.53916493, 1.48603746, 1.87630817, 1.77334621,
       1.64850536, 1.58639298, 1.58941719, 1.79040924, 1.04583909,
       1.74486094, 1.81991222, 1.02395036, 1.81135063, 1.76879997,
       1.70253768, 1.93563904, 1.19357163, 1.57572328, 1.2522905 ,
       1.81195972, 1.32857493, 1.42176691, 1.55268213, 1.92639604,
       1.54548728, 1.67609546, 1.87078246, 1.63240654, 1.28700308,
       1.81390547, 1.67116378, 1.55486981, 1.98191461, 1.37960148,
       1.94297223, 1.5420515 , 1.35617866, 1.31553294, 1.8931597 ,
       1.43638631, 1.38449982, 1.92079378, 1.49271228, 1.49634814,
       1.30202162, 1.44410815, 1.99164158, 1.84213511, 1.88615546,
       1.00088807, 1.7576481 , 1.45144671, 1.82573873, 1.39704878,
       1.52758188, 1.86603976, 1.26710737, 1.2313003 , 1.04582294,
      